word_level

In [ ]:
def align_ner_with_whisper(ner_entities, whisper_words):
    results = []

    for ent in ner_entities:
        ent_start = ent["start"]
        ent_end = ent["end"]
        name = ent["name"]

        # print(ent_start, ent_end)
        matched_words = []

        for word in whisper_words[str(name)]:
            w_start = word["char_start"]
            w_end = word["char_end"]

            # if int(w_start) <= int(ent_start) <= int(w_end):   # 方法1
            #     matched_words.append(word)

            #檢查是否重疊（部分包含）
            if not (int(w_end) <= int(ent_start) or int(w_start) >= int(ent_end)): # 方法2
                matched_words.append(word)






        print(matched_words)


        if matched_words:
            ts_start = matched_words[0]['start_time']
            ts_end = matched_words[-1]['end_time']
            results.append({
                "entity": ent["word"],
                "start_time": ts_start,
                "end_time": ts_end
            })
        else:
            results.append({
                "entity": ent["word"],
                "start_time": None,
                "end_time": None
            })

    return results

In [ ]:
import json

with open("/content/task1_whisper_word_timestamps.json", "r") as f:
    wisper_word = json.load(f)

with open("/content/pre_answer_model_checkpoint-2088_0.txt", "r") as f:
  data = f.readlines()


pre_dict = {}
pre_list = []

for line in data :

  line = line.strip()
  line_split = line.split("\t")


  pre_dict["name"] = line_split[0]
  pre_dict["type"] = line_split[1]
  pre_dict["start"] = line_split[2]
  pre_dict["end"] = line_split[3]
  pre_dict["word"] = line_split[4]

  pre_list.append(pre_dict)
  pre_dict = {}

# print(pre_list)

# print( pre_list[0] )
# print(wisper_word["271.wav"])

# print(wisper_word)

results = align_ner_with_whisper(pre_list, wisper_word)
# print(results)

[{'word': 'July', 'start_time': 2.84, 'end_time': 3.12, 'char_start': 33, 'char_end': 37}, {'word': '19,', 'start_time': 3.12, 'end_time': 3.44, 'char_start': 38, 'char_end': 41}, {'word': '2062', 'start_time': 3.94, 'end_time': 4.96, 'char_start': 42, 'char_end': 46}]
[{'word': 'July', 'start_time': 5.6, 'end_time': 5.88, 'char_start': 51, 'char_end': 55}, {'word': '28,', 'start_time': 5.88, 'end_time': 6.38, 'char_start': 56, 'char_end': 59}, {'word': '2062', 'start_time': 6.86, 'end_time': 7.84, 'char_start': 60, 'char_end': 64}]
[{'word': "Amanda's", 'start_time': 12.7, 'end_time': 13.3, 'char_start': 125, 'char_end': 133}]
[{'word': 'T.', 'start_time': 1.82, 'end_time': 1.98, 'char_start': 17, 'char_end': 19}, {'word': 'Wernz', 'start_time': 2.2, 'end_time': 2.94, 'char_start': 20, 'char_end': 25}]
[{'word': 'June', 'start_time': 5.92, 'end_time': 6.48, 'char_start': 52, 'char_end': 56}, {'word': '6,', 'start_time': 6.48, 'end_time': 7.12, 'char_start': 57, 'char_end': 59}, {'word

In [ ]:
with open("/content/task2_answer_word.txt", "w") as f:
    for index, r in enumerate(pre_list):
        start_time = float(results[index]['start_time'])
        end_time = float(results[index]['end_time'])
        f.write(f"{r['name']}\t{r['type']}\t{start_time:.3f}\t{end_time:.3f}\t{results[index]['entity']}\n")


char_level

In [ ]:
def align_char_level_ner_with_whisper(ner_entity, whisper_chars):

    result = []

    for ner in ner_entity:

      name = ner["name"]

      ent_start = int(ner["start"])
      ent_end = int(ner["end"])

      # print(ent_start, ent_end)



      matched_chars = [
          c for c in whisper_chars[name]
          if ent_start <= c["index"] < ent_end
      ]

      if matched_chars:
          ts_start = matched_chars[0]["start"]
          ts_end = matched_chars[-1]["end"]
          result.append( {
              "entity": ner["word"],
              "start_time": ts_start,
              "end_time": ts_end
          } )
      else:
          result.append( {
              "entity": ner["word"],
              "start_time": None,
              "end_time": None
          } )

    return result


In [ ]:
import json

with open("/content/J_fix_timestamp.json", "r") as f:
    wisper_word = json.load(f)

with open("/content/pre_answer_model_checkpoint-2088_0.txt", "r") as f:
  data = f.readlines()


pre_dict = {}
pre_list = []

for line in data :

  line = line.strip()
  line_split = line.split("\t")


  pre_dict["name"] = line_split[0]
  pre_dict["type"] = line_split[1]
  pre_dict["start"] = line_split[2]
  pre_dict["end"] = line_split[3]
  pre_dict["word"] = line_split[4]

  pre_list.append(pre_dict)
  pre_dict = {}

# print(pre_list)

# print( pre_list[0] )
# print(wisper_word["271"])


result = align_char_level_ner_with_whisper(pre_list, wisper_word)

print( result )

[{'entity': 'July 19, 2062', 'start_time': 3.479, 'end_time': 6.226}, {'entity': 'July 28, 2062', 'start_time': 6.446, 'end_time': 9.313}, {'entity': 'Amanda', 'start_time': 14.345, 'end_time': 14.786}, {'entity': 'T. Wernz', 'start_time': 2.758, 'end_time': 3.761}, {'entity': 'June 6, 2063', 'start_time': 8.053, 'end_time': 12.284}, {'entity': 'June 6, 2063', 'start_time': None, 'end_time': None}, {'entity': 'Catherine', 'start_time': 0.593, 'end_time': 1.014}, {'entity': 'Mars', 'start_time': 4.323, 'end_time': 4.504}, {'entity': 'Catherine', 'start_time': 11.103, 'end_time': 11.564}, {'entity': 'Kerr', 'start_time': 14.312, 'end_time': 14.493}, {'entity': 'Martin', 'start_time': 14.994, 'end_time': 15.456}, {'entity': 'CM', 'start_time': 2.235, 'end_time': 2.375}, {'entity': 'May 24, 2063', 'start_time': 2.636, 'end_time': 5.641}, {'entity': 'Z. Wisnowski', 'start_time': 9.548, 'end_time': 10.389}, {'entity': '74M-52173', 'start_time': 0.419, 'end_time': 0.84}, {'entity': 'Sarah', '

In [ ]:
with open("/content/task2_answer_char.txt", "w") as f:

  for index, r in enumerate(pre_list):
    f.write(f"{r['name']}\t{r['type']}\t{result[index]['start_time']}\t{result[index]['end_time']}\t{result[index]['entity']}\n")